# SURAJ PATI | 21EC39025
---
# Q1. Read .bmp Image:
---

In [5]:
def read_bmp_image(filename):
    with open(filename, "rb") as f:
        signature = f.read(2).decode("ascii")
        if signature != "BM":
            return None

        file_size = int.from_bytes(f.read(4), "little")
        reserved = int.from_bytes(f.read(4), "little")
        data_offset = int.from_bytes(f.read(4), "little")

        info_header_size = int.from_bytes(f.read(4), "little")
        width = int.from_bytes(f.read(4), "little")
        height = int.from_bytes(f.read(4), "little")
        planes = int.from_bytes(f.read(2), "little")
        bits_per_pixel = int.from_bytes(f.read(2), "little")
        compression = int.from_bytes(f.read(4), "little")
        image_size = int.from_bytes(f.read(4), "little")
        XpixelsperM = int.from_bytes(f.read(4), "little")
        YpixelsperM = int.from_bytes(f.read(4), "little")
        colors_used = int.from_bytes(f.read(4), "little")
        imp_colors = int.from_bytes(f.read(4), "little")

        color_table = []
        if bits_per_pixel <= 8:
            for x in range(colors_used):
                red = int.from_bytes(f.read(1), "little")
                green = int.from_bytes(f.read(1), "little")
                blue = int.from_bytes(f.read(1), "little")
                reserved = int.from_bytes(f.read(1), "little")
                color_table.append((red, green, blue, reserved))

        image_array = []
        for x in range(height):
            row = []
            for y in range(width):
                row.append(int.from_bytes(
                    f.read(bits_per_pixel // 8), "little"))
            image_array.append(row)

        return {
            "filename": filename,
            "file_size": file_size,
            "data_offset": data_offset,
            "width": width,
            "height": height,
            "bits_per_pixel": bits_per_pixel,
            "image_size": image_size,
            "color_table": color_table,
            "image_array": image_array
        }

# Q2. Write .bmp Image:
---

In [6]:
def write_bmp(image_data, filename):
    with open(filename, "wb") as f:
        signature = "BM"
        data_offset = 14 + 40 + \
            (4 * len(image_data["color_table"])
             if image_data["bits_per_pixel"] <= 8 else 0)
        file_size = data_offset + \
            image_data["height"] * image_data["width"] * \
            image_data["bits_per_pixel"] // 8
        reserved = 0

        f.write(signature.encode("ascii"))
        f.write(file_size.to_bytes(4, "little"))
        f.write(reserved.to_bytes(4, "little"))
        f.write(data_offset.to_bytes(4, "little"))

        f.write((40).to_bytes(4, "little"))
        f.write(image_data["width"].to_bytes(4, "little"))
        f.write(image_data["height"].to_bytes(4, "little"))
        f.write((1).to_bytes(2, "little"))
        f.write(image_data["bits_per_pixel"].to_bytes(2, "little"))
        f.write((0).to_bytes(4, "little"))
        f.write(image_data["image_size"].to_bytes(4, "little"))
        f.write((0).to_bytes(4, "little"))
        f.write((0).to_bytes(4, "little"))
        f.write(len(image_data["color_table"]).to_bytes(4, "little"))
        f.write((0).to_bytes(4, "little"))

        if image_data["bits_per_pixel"] <= 8:
            for color in image_data["color_table"]:
                f.write(color[0].to_bytes(1, "little"))
                f.write(color[1].to_bytes(1, "little"))
                f.write(color[2].to_bytes(1, "little"))
                f.write(color[3].to_bytes(1, "little"))

        for row in image_data["image_array"]:
            for pixel in row:
                f.write(pixel.to_bytes(
                    image_data["bits_per_pixel"] // 8, "little"))

# Q3. Color Channel Manipulation:
---

In [7]:
def remove_color(image_data, color_index, filename):
    copy_color_table = image_data["color_table"].copy()
    for i in range(len(image_data["color_table"])):
        image_data["color_table"][i] = list(image_data["color_table"][i])
        image_data["color_table"][i][color_index] = 0
    write_bmp(image_data, filename)
    image_data["color_table"] = copy_color_table

# Results:
---


In [10]:
image_filenames = ["cameraman.bmp", "corn.bmp", "pepper.bmp"]
count = 0

for img in image_filenames:
    Image = read_bmp_image(img)
    if Image:
        print(f"  Image: {img}")
        print(f"  Width: {Image['width']} pixels")
        print(f"  Height: {Image['height']} pixels")
        print(f"  Bits per Pixel: {Image['bits_per_pixel']}")
        print(f"  Image Size: {Image['image_size']} bytes")
        print(f"  Colors Used: {len(Image['color_table'])} colors")
        print(f"  Pixel Array of {img}: {(Image['image_array'])}")
        print("\n")

        output_filename = img.replace(".bmp", "_written_output.bmp")
        write_bmp(Image, output_filename)

        if img == "corn.bmp":
            remove_color(Image, 2, "corn_filter_red_channel.bmp")
            remove_color(Image, 0, "corn_filter_blue_channel.bmp")
            remove_color(Image, 1, "corn_filter_green_channel.bmp")

  Image: cameraman.bmp
  Width: 256 pixels
  Height: 256 pixels
  Bits per Pixel: 8
  Image Size: 65536 bytes
  Colors Used: 256 colors
  Pixel Array of cameraman.bmp: [[233, 211, 216, 192, 188, 211, 192, 226, 203, 174, 205, 191, 197, 226, 205, 202, 185, 217, 199, 197, 211, 211, 214, 217, 216, 197, 173, 180, 211, 185, 81, 100, 124, 112, 137, 136, 142, 135, 128, 111, 89, 80, 82, 104, 122, 125, 114, 137, 143, 135, 118, 183, 208, 198, 183, 178, 188, 182, 184, 191, 178, 225, 162, 54, 53, 52, 51, 66, 92, 113, 99, 125, 145, 156, 156, 165, 159, 156, 161, 170, 151, 77, 60, 130, 198, 193, 215, 200, 219, 199, 189, 182, 211, 211, 195, 205, 196, 200, 218, 200, 156, 115, 145, 133, 208, 220, 246, 252, 255, 243, 182, 213, 191, 208, 208, 211, 240, 192, 190, 214, 212, 168, 179, 185, 186, 217, 205, 209, 192, 217, 203, 205, 215, 212, 228, 185, 187, 217, 181, 218, 204, 198, 187, 191, 189, 177, 193, 210, 170, 160, 200, 179, 173, 184, 181, 182, 182, 178, 186, 198, 184, 189, 212, 180, 226, 205, 170, 177, 202